In [5]:
print(cwfid_path)

/Users/hlyd4326/Documents/sih_projects/Weed-ID-Interchange/conversion_tools/cwfid_to_json


In [1]:
# %load_ext nb_black

import yaml
import pandas as pd
import pathlib
import uuid
import json
import time
import datetime
from tqdm import tqdm
import humanfriendly
import exifread
import sys
import os
import PIL
from PIL import Image

cwfid_path = pathlib.Path(os.path.realpath('../cwfid_to_json/'))

input_metadata_file = cwfid_path / "labels.csv"
output_file = cwfid_path / "cwfid_imageinfo.json"
image_folder = cwfid_path / "cwfid_images"
dir_name = "cwfid_images"

In [11]:

CATEGORY_MAP = {
    # TODO: we need a spec for species unspecified, and we need a spec for species specified
    'crop': {'common_name': 'carrot',
             'species': 'daugus carota',
             'eppo_taxon_code':'DAUCS',
             'eppo_nontaxon_code':'3UMRC',
             'role': 'crop',
             'id': 0},
    'weed': {'species': 'UNSPECIFIED',
             'role': 'weed',
             'id': 1}
}

def create_annotations(ann_blob, image_id, starting_idx):
    annotations = []
    for i, obj in enumerate(ann_blob['annotation'], starting_idx):
        category = CATEGORY_MAP[obj['type']]
        # a COCO polygon is just a sequence [[x1, y1, x2, y2, ...]]
        if not isinstance(obj["points"]["x"], list):
            print(f"Found invalid polygon for annotation of {ann_blob['filename']} with points {obj['points']}")
            continue
        polygon = zip(obj["points"]["x"], obj["points"]["y"])
        polygon = sum(polygon, ())  # flatten an iterable of tuples
        polygon = [list(map(int, polygon))]
        # TODO: parse this polygon with pycocotools.frPyObjects(polygon, im_height, im_width)
        # then:
        # * check that mask matches the stored images in cwfid, and that there is not an off-by-one error (due to indexing base)
        # * get area and bbox for annotation object
        annotations.append({
            "id": i,
            "image_id": image_id,
            "category_id": category["id"],
            "segmentation": polygon,
            "iscrowd": 0,
            # something in pycocotools to do this
            # "area": calculate_area(polygon),
            # "bbox": ...
        })
    return annotations

In [12]:
def get_image_dimensions(path):
    if not os.path.isfile(path):
        return None
    # Retrieve image width and height
    image = PIL.Image.open(path)
    width, height = image.size
    # Calculate resolution in pixels
    resolution =  width*height
    return {'width': width, 'height': height, 'resolution': resolution}

In [13]:
missing_files = []
categories = [
    CATEGORY_MAP['crop'],
    CATEGORY_MAP['weed']
]
agcontext = [{"id": 0,
              "agcontext_name":"cwfid",
              "crop_type":"root vegetable",
              "upload_time":datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
              "camera_type": "TODO"
          }]
license = [{"id": 0,
           "license_name":"Creative Commons Attribution 4.0",
           "license_version": "4.0",
           "url":"https://creativecommons.org/licenses/by/4.0/"}]
info = {}  # TODO
annotations = []
images = []
progress = tqdm((cwfid_path / "annotations").glob("*_annotation.yaml"))
for ann_path in progress:
    progress.set_description(ann_path.name)
    image_id = int(ann_path.name[:3])
    ann_blob = yaml.safe_load(ann_path.open())
    
    image = {
        "id": image_id,
        "file_name": os.path.join(dir_name, ann_blob["filename"]),
        "license": 0,  # TODO
        "agdata_id": 0,
    }
    dims = get_image_dimensions(image_folder / ann_blob["filename"])
    
    if dims is None:
        missing_files.append(image_folder / ann_blob["filename"])
    else:
        image.update(dims)
        
    images.append(image)
    
    annotations.extend(create_annotations(ann_blob, image_id, starting_idx=len(annotations)))


008_annotation.yaml: : 13it [00:00, 53.51it/s]

Found invalid polygon for annotation of 018_image.png with points {'x': 685.0, 'y': 200.0}
Found invalid polygon for annotation of 009_image.png with points {'x': 457.0, 'y': 650.0}


020_annotation.yaml: : 60it [00:00, 69.21it/s]


In [14]:
collections = [
    # TODO: DCMI conformance
    {"author": "Haug, Sebastian and Ostermann, Jörn",
     "title": "A Crop/Weed Field Image Dataset for the Evaluation of Computer Vision Based Precision Agriculture Tasks",
     "year": 2015,
     "identifier": "doi:10.1007/978-3-319-16220-1_8",
     "rights": "All data is subject to copyright and may only be used for non-commercial research. In case of use please cite our publication.",
     "accrual_policy": "Closed",
     "id": 0,
    }
]

In [15]:
subset_path = cwfid_path / "train_test_split.yaml"
with open(subset_path) as subset_file:
    subsets = yaml.safe_load(subset_file)
    
    # TODO: iterate over value key pairs and create individual collection membership per image.
    # TODO: refer to all annotations associated with an image?
    collection_memberships = []
    for key, value in subsets.items():
        if key == 'train':
            collection_memberships.append({
                "image_id": value,
                "subset": "train",
                "collection_id": 0
            }) 
        else:
            collection_memberships.append({
                "image_id": value,
                "subset": "test",
                "collection_id": 0
            }) 
print(collection_memberships)


[{'image_id': [2, 5, 6, 7, 8, 11, 12, 14, 16, 17, 18, 19, 20, 23, 24, 25, 27, 28, 31, 33, 34, 36, 37, 38, 40, 41, 42, 43, 45, 46, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59], 'subset': 'train', 'collection_id': 0}, {'image_id': [1, 3, 4, 9, 10, 13, 15, 21, 22, 26, 28, 29, 30, 32, 35, 39, 44, 47, 48, 54, 60], 'subset': 'test', 'collection_id': 0}]


In [16]:
"""Write output"""
with output_file.open('w') as fout:
    json.dump({
               "images": images,
               "annotations": annotations,
               "categories": categories,
               "info": info,
               "license":license,
               "agcontext": agcontext,
               "collections": collections,
               "collection_memberships": collection_memberships},
              fout, indent=4)

In [46]:
!pbcopy < $output_file